<a href="https://colab.research.google.com/github/ersilia-os/event-fund-ai-drug-discovery/blob/main/session2_breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ersilia Model Hub
The Ersilia Model Hub is a repository of pre-trained, ready-to-use AI models for drug discovery. A list of models and its applications is available [here](https://ersilia.io/model-hub).

You can run the Ersilia Model Hub in your computer by installing the [Ersilia Python Package](https://github.com/ersilia-os/ersilia). In this session, we will use the Google Colab implementation of the Ersilia Model Hub to ensure compatibility with all systems. 


In [1]:
# Installation on Google Colaboratory
%%capture
%env MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.12.0-Linux-x86_64.sh
%env MINICONDA_PREFIX=/usr/local
%env PYTHONPATH={PYTHONPATH}:/usr/local/lib/python3.7/site-packages
%env CONDA_PREFIX=/usr/local
%env CONDA_PREFIX_1=/usr/local
%env CONDA_DEFAULT_ENV=base
!pip install git+https://github.com/ersilia-os/ersilia.git
!wget https://repo.anaconda.com/miniconda/$MINICONDA_INSTALLER_SCRIPT
!chmod +x $MINICONDA_INSTALLER_SCRIPT
!./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
!python -m pip install flask
!python -m pip install urllib<=1.25
!python -m pip install chardet<=5.0
import sys
_ = (sys.path.append("/usr/local/lib/python3.7/site-packages"))

## Breakout activity

We will use the Ersilia Model Hub to filter a list of 400 compounds (the MMV Pathogen Box) to try and identify the best ones for experimental testing.

We will use our directory on Google Drive to store predictions.

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/DataScience_Workshop
!git clone https://github.com/ersilia-os/event-fund-ai-drug-discovery.git /content/drive/MyDrive/DataScience_Workshop

Mounted at /content/drive
Cloning into '/content/drive/MyDrive/DataScience_Workshop'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 84 (delta 33), reused 53 (delta 19), pack-reused 0
Unpacking objects: 100% (84/84), done.


There are three basic commands to run a model with Ersilia:
1.   Fetch: retrieves a model from its online storage
2.   Serve: brings the model alive, ready to be used
3.   Predict: makes predictions for any input molecules of interest

More details are available in the Ersilia Model Hub [documentation](https://ersilia.gitbook.io/ersilia-book/).



In [2]:
!ersilia fetch eos2gth

⬇️  Fetching model eos2gth: maip-malaria-surrogate
👍 Model eos2gth fetched successfully!


In [3]:
!ersilia serve eos2gth

🚀 Serving model eos2gth: maip-malaria-surrogate

   URL: http://127.0.0.1:40415
   PID: 4245
   SRV: conda

👉 Available APIs:
   - predict


In [7]:
!ersilia api predict -i "drive/MyDrive/DataScience_Workshop/data/test_drugs.csv" -o "drive/MyDrive/DataScience_Workshop/data/eos2gth.csv"

drive/MyDrive/DataScience_Workshop/data/eos2gth.csv


We will fetch the different models we can use for the exercise:


*   Malaria Activity
*   Tuberculosis Activity
*   Antibiotic Activity
*   Cardiotoxicity (hERG)
*   Natural Product Likeness
*   Retrosynthetic Accessibility
*   Aqueous Solubility

